# Segmenting and Clustering Neighborhoods in Toronto

## Requirements

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.  

1. Start by creating a new Notebook for this assignment.  

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe. 

3. To create the above dataframe:
    * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Scrap Wikipedia for data using BeautifulSoup

In [2]:
#Install necessary packages for Web Scaping
!pip install BeautifulSoup4
!pip install requests

##### Step 1: Get Data
1. Get HTML from Wikipedia
2. Parse html data by using BeautifySoup
3. Store parsed data into Pandas DataFrame

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

wiki_raw_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(wiki_raw_data, 'html.parser')

#Use beautiful_soup, iterate the class .wikitable to get the data from the HTML page and then store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())

    # First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#Convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###### Step 2: Data Cleanup and re-grouping
1. Process only cell that have an assigned borough and Ignore cells with a borough that are 'Not assigned'
2. Sort and Reset index
3. If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough
4. More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma

In [4]:
# Only process the cells that have an assigned borough. 
# Ignore cells with a borough that are Not assigned.
canada_df = canada_df[~canada_df['Borough'].isin(['Not assigned'])]

# Sort and Reset index.
canada_df = canada_df.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
canada_df.loc[canada_df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = canada_df['Borough']

# More than one neighborhood can exist in one postal code area.  
# These two rows will be combined into one row with the neighborhoods separated with a comma.
canada_df = canada_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
canada_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


###### Step 3: Display the final table

In [6]:
canada_df.shape

(103, 3)

In [7]:
canada_df.to_csv('Toronto_FSAs.csv', index=False)